# TP3 Clasificación de clubes del FIFA

En este trabajo vot a demostrar que puedo identificar a qué club pertenece un jugador según sus características. Si bien en este caso voy a utilizar 2 clubes que sean muy diferentes entre ellos, para facilitar la ejemplificación y llegar a resultados satisfactorios, tambien se puede adaptar para distinguir entre grupos de clubes y ligas, o si un jugador podría pertenecer a un club de rango más alto, ahí es donde hay que usar la imaginación y pensar en las soluciones para las necesidades reales.

Fuente kaggle: https://www.kaggle.com/datasets/mukeshmanral/fifa-data-for-eda-and-stats

# EDA (Analisis Exploratorio de Datos)

## Dataset

In [32]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [33]:
df = pd.read_csv("Datos/fifa_eda_stats.csv")
df.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,Right,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,Right,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,Right,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


### Filtramos algunos clubes diferentes entre si

In [34]:
df = df[df['Club'].isin(df.Club.unique()[[0,100,150,250,350,250,550]])]

In [35]:
df.Club.unique()

array(['FC Barcelona', 'RCD Espanyol', 'Melbourne Victory',
       'Kaizer Chiefs', 'CD Aves', 'Östersunds FK'], dtype=object)

### Filtro solo variables numéricas

In [36]:
df2 = df.select_dtypes(['number'])

In [37]:
df2['Club'] = df['Club']
df2

,ID,Age,Overall,Potential,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,Finishing,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Club
0,158023,31,94,94,5.0,4.0,4.0,10.0,84.0,95.0,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,FC Barcelona
7,176580,31,91,91,5.0,4.0,3.0,9.0,77.0,93.0,...,85.0,62.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,FC Barcelona
18,192448,26,89,92,3.0,4.0,1.0,22.0,15.0,14.0,...,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,FC Barcelona
20,189511,29,89,89,4.0,3.0,3.0,5.0,62.0,67.0,...,90.0,90.0,86.0,80.0,5.0,8.0,13.0,9.0,13.0,FC Barcelona
32,189242,26,88,89,3.0,4.0,5.0,7.0,79.0,79.0,...,85.0,55.0,54.0,47.0,12.0,7.0,9.0,14.0,6.0,FC Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17116,245597,18,55,73,1.0,2.0,2.0,33.0,51.0,23.0,...,42.0,56.0,48.0,46.0,11.0,9.0,12.0,12.0,6.0,RCD Espanyol
17316,241440,18,54,69,1.0,3.0,2.0,19.0,38.0,52.0,...,55.0,32.0,27.0,34.0,12.0,8.0,15.0,8.0,12.0,Melbourne Victory
17541,243729,18,53,67,1.0,2.0,2.0,26.0,50.0,45.0,...,45.0,24.0,22.0,23.0,15.0,14.0,8.0,7.0,12.0,Melbourne Victory
18027,205861,24,50,55,1.0,2.0,1.0,31.0,20.0,14.0,...,30.0,13.0,16.0,17.0,51.0,46.0,45.0,49.0,51.0,Östersunds FK


### Elimino los valores nulos (nan)

In [38]:
df2 = df2.dropna()

In [39]:
df2.shape

(160, 43)

## Matríz de Correlación

La matriz de correlación nos indica que tanta influencia tienen los valores de las columnas con respecto al resto. 

Nota: Ojo con las correlaciones negativas, a veces pueden ser inversamente proporcionales y son igual de influentes que las positivas.

In [40]:
# Convierto el target en numeros para poder graficar
df2['ClubN'] = df2.Club.replace(df2.Club.unique(), range(df2.Club.unique().shape[0],0,-1))

In [41]:
import plotly.express as px

m_corr = df2.corr()
fig = px.imshow(m_corr, x=m_corr.columns, y=m_corr.columns,width=800, height=800)
fig.update_xaxes(nticks=m_corr.shape[1])
fig.update_yaxes(nticks=m_corr.shape[1])
fig.show()

## Selección de variables

In [42]:
# Me quedo con las varibles que tienen más influencia sobre el target
X = df2[['Overall','Potential','International Reputation', 'ShortPassing','LongPassing','Reactions','Vision','Composure']]

## Gráficos de puntos para comparación de variables 

Para facilitar la compresión de como nuestro modelo de ML podría dividir en las categorias esperada, vamos a crear graficos cruzando variables y coloreando por el target correspondiente

In [43]:
# Calculo la cantidad de gráficos: para abarcar todas las posibles combinaciones de variables
# utilizamos la siguiente sucesión comb(n) = n-1 + comb(n-1)
def comb(n): 
  if n == 1: 
    return 0 
  else: 
    return n-1 + comb(n-1)

nFeatures = X.shape[1]

nGraph = comb(nFeatures)
nGraph

28

In [44]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# filas necesarias para que entren 5 gráficos en cada una de ellas (ceil redondea para arriba)
nRows = math.ceil( nGraph / 5)

# Configuración del layout donde agregaremos los gráficos
fig = make_subplots(rows=nRows, cols=5, subplot_titles=[' Title Ej '] * nGraph)

# Variables complementarias
r = 1
c = 0
titles =  []
nh = 0

for fh in X.columns[:-1]:
  nh += 1
  for fv in X.columns[nh:]:
    if c < 5:
      c += 1
    else:
      c = 1
      r +=1
    
    titles.append(fh + ' vs ' + fv)

    fig.add_trace(
        go.Scatter(x=X[fh],y=X[fv],mode="markers",
        marker=dict(
            color=df2.ClubN, #set color equal to a variable
        ),showlegend=False),
        row=r,
        col=c
    )

# Actualizo los titulos de los gráficos
for i in range(len(titles)):
  fig.layout.annotations[i].update(text=titles[i], font={"size":8})

# Le doy un alto de 200 por fila
fig.update_layout(height=200*nRows)
fig.show()

### Creo la variable y

In [45]:
y = df2['ClubN']

## Normalización

In [46]:
transformer = Normalizer().fit_transform(X)
transformer

array([[0.3823445 , 0.3823445 , 0.02033747, ..., 0.386412  , 0.3823445 ,
        0.39047949],
       [0.40634719, 0.40634719, 0.02232677, ..., 0.41081255, 0.37508972,
        0.37955507],
       [0.46776715, 0.48353459, 0.01576743, ..., 0.44674391, 0.36265094,
        0.36265094],
       ...,
       [0.40952595, 0.51770262, 0.0077269 , ..., 0.27816857, 0.40179905,
        0.34771071],
       [0.50282374, 0.55310612, 0.01005647, ..., 0.46259784, 0.11062122,
        0.30169425],
       [0.44277272, 0.62873726, 0.00885545, ..., 0.38963999, 0.27451908,
        0.37192908]])

## División de los datos

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [48]:
Valx_train, Valx_test, Valy_train, Valy_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Modelado

In [50]:
from sklearn.svm import LinearSVC
penalty = ['l1','l2']
c = np.arange(1,5,1)
Result = pd.DataFrame(columns=['Penalty','C','Acurracy'])
for penal in penalty:
    for cant in c:
        modelo= LinearSVC(loss='squared_hinge',penalty=penal,C=cant,dual=False,max_iter=800000)
        modelo.fit(Valx_train, Valy_train)
        modelo.predict(Valx_test)
        Acurracy = modelo.score(Valx_test, Valy_test)
        Result.loc[len(Result)] = [penal, cant, Acurracy]

Debe encontrar el modelo con los hiperparametros que den mejores resultados. Probar con SVC, LinearSVC y SGDClassifier. Observar si su modelo está overfitiando o underfitiando. Redactar una conclución.

Como yo fuera del proyecto final que presento ya hice una evaluacion con los otros modelos y llegue a la conclucion que LinearSVC es el mejor, es el que dejo para presentar.

In [51]:
Result

,Penalty,C,Acurracy
0,l1,1,0.305556
1,l1,2,0.333333
2,l1,3,0.333333
3,l1,4,0.361111
4,l2,1,0.305556
5,l2,2,0.277778
6,l2,3,0.250000
7,l2,4,0.277778


## Entrenamiento y evaluación

In [53]:
from sklearn.svm import LinearSVC
modelo= LinearSVC(loss='squared_hinge',penalty='l1',C=4,dual=False,max_iter=800000)
modelo.fit(X_train, y_train)
modelo.predict(X_test)
modelo.score(X_test, y_test)

0.5849056603773585